In [73]:
import nltk
import re
import pickle
from sklearn.datasets import load_files
from nltk.stem.porter import *

In [74]:
def simplify(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A).lower().strip() # Remove special characters, whitespaces and make lower case
    tokens = nltk.WordPunctTokenizer().tokenize(doc) # Tokenize
    filtered_tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')] # Remove stopwords
    doc = ' '.join(filtered_tokens) # Re-create document from filtered tokens
    return doc

In [75]:
def get_words(sentence):
    stemmer = PorterStemmer()
    words = [stemmer.stem(x) for x in simplify(sentence).split()]
    return words

In [23]:
#dataset = load_files('reviews')
#features, labels = dataset.data, dataset.target

In [30]:
# with open('features.pickle', 'wb') as f:
#     pickle.dump(features, f)
# with open('labels.pickle', 'wb') as f:
#     pickle.dump(labels, f)

In [31]:
# Loading dataset (Unpickling datasets)
with open('features.pickle', 'rb') as f:
    features = pickle.load(f)
    
with open('labels.pickle', 'rb') as f:
    labels = pickle.load(f)

In [76]:
word_sentiment = {}

In [77]:
for i in range(len(features)):
    sentiment, line = labels[i],str(features[i])
    words = get_words(line)
    for word in words:
        try: # Increment weight of the word by 1 if positive, decrement weight by 1 if negative
            word_sentiment[word][0] = word_sentiment[word][0] + 1 if int(sentiment) == 1 else word_sentiment[word][0] - 1
        except: # If word doesn't exist, create new entry in dictionary
            word_sentiment[word] = [1, 0] if int(sentiment) == 1 else [-1, 0]
        finally: # Increment number of occurences of the word; used later to compute the weighted sum instead of just sum
            word_sentiment[word][1] += 1

In [78]:
word_weighted_sentiment = {word: word_sentiment[word][0] / word_sentiment[word][1] for word in word_sentiment.keys()}
# Weighted sum of the sentiment of each word (divide weight of the word by number of occurences of the word)

In [83]:
text_to_sentiment = "bad film"

In [84]:
sentiment = 0
words = get_words(text_to_sentiment)
for word in words:
    try:
        sentiment += word_weighted_sentiment[word] # Compute sum of weighted sentiments of the words in the review
    except:
        sentiment += 0 # MISSING WORDS (words in testing data but not training data) ARE GIVEN SENTIMENT OF 0
if sentiment >= 0:
    print("Possitive")
else:
    print("Negative")

Negative
